In [1]:
import sqlalchemy
sqlalchemy.__version__  

'2.0.31'

Python DBAPI 是 SQLAlchemy 用于与特定数据库交互的第三方驱动程序。在这种情况下，我们使用名称 pysqlite，在现代 Python 用法中，它是 SQLite 的 sqlite3 标准库接口。

我们的 URL 包含短语 /:memory:，这是向 sqlite3 模块指示我们将使用仅限内存数据库的指示符。这种数据库非常适合进行实验，因为它不需要任何服务器，也不需要创建新文件。

当 Engine 最初由 create_engine() 返回时，它实际上尚未尝试连接到数据库；只有在第一次要求它对数据库执行任务时才会发生这种情况。这是一种称为 延迟初始化 的软件设计模式。

我们还指定了一个参数 create_engine.echo，它将指示 Engine 将其发出的所有 SQL 记录到一个 Python 日志记录器中，该日志记录器将写入标准输出。此标志是设置 Python 日志记录的正式方式 的一种简便方法，并且对于在脚本中进行实验很有用。

In [2]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

我们现在可以深入研究 Engine 的基本操作及其主要交互端点，即 Connection 和 Result。

在使用 ORM 时，Engine 由另一个名为 Session 的对象管理。现代 SQLAlchemy 中的 Session 强调了一种事务性和 SQL 执行模式，与下面讨论的 Connection 的模式基本相同，因此，虽然本节主要针对 Core，但这里的所有概念实际上也与 ORM 使用相关

上下文管理器提供了数据库连接，并将操作封装在事务中。Python DBAPI 的默认行为包括始终进行事务；当连接的范围 释放 时，将发出 ROLLBACK 以结束事务。事务 **不会自动提交**；当我们想要提交数据时，通常需要调用 Connection.commit()

In [3]:
from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2024-06-30 16:05:15,145 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:15,146 INFO sqlalchemy.engine.Engine select 'hello world'
2024-06-30 16:05:15,147 INFO sqlalchemy.engine.Engine [generated in 0.00187s] ()
[('hello world',)]
2024-06-30 16:05:15,148 INFO sqlalchemy.engine.Engine ROLLBACK


In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text(
            "INSERT INTO some_table (x, y) VALUES (:x, :y)"
        ),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2024-06-30 16:05:15,397 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:15,398 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2024-06-30 16:05:15,399 INFO sqlalchemy.engine.Engine [generated in 0.00183s] ()
2024-06-30 16:05:15,400 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-06-30 16:05:15,401 INFO sqlalchemy.engine.Engine [generated in 0.00071s] [(1, 1), (2, 4)]
2024-06-30 16:05:15,402 INFO sqlalchemy.engine.Engine COMMIT


还有一种提交数据的方式，即我们可以预先将“连接”块声明为事务块。对于这种操作模式，我们使用 Engine.begin() 方法获取连接，而不是使用 Engine.connect() 方法。此方法将同时管理 Connection 的范围，并将所有内容包含在一个事务中，并在成功执行块后执行 COMMIT，或在出现异常时执行 ROLLBACK。这种方式被称为 **一次开始**

输出中的的“BEGIN (implicit)”意味着 SQLAlchemy **实际上没有向数据库发送任何命令**；

In [5]:
# 使用参数防止SQL注入
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        # 数组表示要多次调用同一个SQL并使用数组中的参数
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    
    conn.commit()

2024-06-30 16:05:15,802 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:15,803 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2024-06-30 16:05:15,804 INFO sqlalchemy.engine.Engine [generated in 0.00192s] (2,)
x: 2  y: 4
2024-06-30 16:05:15,805 INFO sqlalchemy.engine.Engine ROLLBACK
2024-06-30 16:05:15,807 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:15,807 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-06-30 16:05:15,808 INFO sqlalchemy.engine.Engine [cached since 0.4084s ago] [(11, 12), (13, 14)]
2024-06-30 16:05:15,809 INFO sqlalchemy.engine.Engine COMMIT


Result 具有许多用于获取和转换行的方法，例如前面说明的 Result.all() 方法，该方法返回所有 Row 对象的列表。它还实现了 Python 迭代器接口，以便我们可以直接迭代 Row 对象的集合。

“execute” 和 “executemany” 之间的一个关键行为差异是，后者不支持返回结果行

In [6]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    # for x, y in result:
    # for dict_row in result.mappings() => dict_row["x"]
    for row in result:
        # x = row[0], y = row[1]
        print(f"x: {row.x}  y: {row.y}")

2024-06-30 16:05:16,258 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:16,259 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-06-30 16:05:16,260 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ()
x: 1  y: 1
x: 2  y: 4
x: 11  y: 12
x: 13  y: 14
2024-06-30 16:05:16,261 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

2024-06-30 16:05:16,619 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:16,620 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2024-06-30 16:05:16,621 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (6,)
x: 11  y: 12
x: 13  y: 14
2024-06-30 16:05:16,622 INFO sqlalchemy.engine.Engine ROLLBACK
2024-06-30 16:05:16,623 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:16,624 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2024-06-30 16:05:16,625 INFO sqlalchemy.engine.Engine [generated in 0.00095s] [(11, 9), (15, 13)]
2024-06-30 16:05:16,626 INFO sqlalchemy.engine.Engine COMMIT


当Session结束事务后，它实际上不会保留Connection对象。它会在下次需要对数据库执行 SQL 时，从Engine获取一个新的Connection

SQLAlchemy Core 和 ORM 的核心元素是 SQL 表达式语言，它允许以流畅、可组合的方式构建 SQL 查询。这些查询的基础是表示数据库概念（如表和列）的 Python 对象。这些对象统称为 数据库元数据。

要开始使用 SQLAlchemy 表达式语言，我们需要构建表示我们想要操作的所有数据库表的 Table 对象。Table 是以编程方式构建的，可以通过直接使用 Table 构造函数，也可以通过使用 ORM Mapped 类间接构建。还可以选择从现有数据库加载部分或全部表信息，称为 反射。

无论使用哪种方法，我们总是从一个集合开始，这个集合将是我们放置**称为 MetaData 对象的表的集合**。这个对象本质上是一个围绕 Python 字典的 facade，它**存储一系列以字符串名称为键的 Table 对象**。虽然 ORM 提供了一些关于从哪里获取这个集合的选项，但我们始终可以选择直接创建一个。一旦我们有了 MetaData 对象，我们就可以声明一些 Table 对象。同一个 MetaData 在 ORM 和 Core 声明的 Table 对象之间共享。

In [8]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String
metadata_obj = MetaData()

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

当我们想要编写引用数据库中 user_account 表的代码时，我们将使用 user_table Python 变量来引用它

 Column 对象在父 Table 中的集合通常通过位于 Table.c 的关联数组访问。

In [9]:
print(user_table.c.name)
print(user_table.c.keys())
print(user_table.primary_key)

user_account.name
['id', 'name', 'fullname']
PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))


最常显式声明的约束是 ForeignKeyConstraint 对象，它对应于数据库 外键约束。当我们声明相互关联的表时，SQLAlchemy 使用这些外键约束声明的存在，不仅是为了在 CREATE 语句中将它们发送到数据库，而且是为了帮助构建 SQL 表达式。

In [10]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

利用表结构建表

In [11]:
metadata_obj.create_all(engine)

2024-06-30 16:05:19,793 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:19,794 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-06-30 16:05:19,795 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:19,796 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-06-30 16:05:19,797 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:19,798 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-06-30 16:05:19,798 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:19,799 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-06-30 16:05:19,800 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:19,801 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-06-30 16:05:19,802 INFO sqlalchemy.engine.Engine [no key 0.00068s] ()
2024-06-30 16:05:19,804 INFO sqlalchemy.engine.Engine 
C

声明 Table 元数据的过程通常与声明 映射 类相结合。映射类是我们想要创建的任何 Python 类，它将具有与数据库表中的列关联的属性。

当使用 ORM 时，MetaData 集合仍然存在，但它本身与一个仅限 ORM 的结构相关联，通常被称为 **声明式基类**。获取新的声明式基类的最便捷方式是创建一个新的类，该类是 SQLAlchemy DeclarativeBase 类的子类。

In [12]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [13]:
Base.metadata

MetaData()

In [14]:
# 它是 SQLAlchemy ORM 中的中心“映射配置”单元。虽然很少直接访问，但此对象是映射配置过程的核心，因为一组 ORM 映射类将通过此注册表相互协调
Base.registry

In [15]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

如果我们没有声明自己的 __init__() 方法，这些类将自动获得一个 __init__() 方法。此方法的默认形式接受所有属性名称作为可选关键字参数。

In [16]:
sandy = User(name="sandy", fullname="Sandy Cheeks")

In [17]:
Base.metadata.create_all(engine)

2024-06-30 16:05:23,688 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:23,689 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-06-30 16:05:23,690 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:23,691 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-06-30 16:05:23,691 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:23,692 INFO sqlalchemy.engine.Engine COMMIT


如何从现有数据库中自动生成Table对象

In [18]:
# 给出表的名称和它将所属的MetaData集合，然后不是指定单个Column和Constraint对象，而是使用Engine传递目标Table.autoload_with参数
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2024-06-30 16:05:24,505 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:24,506 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2024-06-30 16:05:24,506 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:24,508 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2024-06-30 16:05:24,508 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2024-06-30 16:05:24,510 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2024-06-30 16:05:24,510 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:24,511 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2024-06-30 16:05:24,512 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-30 16:05:24,513 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [19]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

### INSERT

In [20]:
from sqlalchemy import insert, select
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

In [21]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [22]:
compiled = stmt.compile()

In [23]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [24]:
compiled.string

'INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)'

In [25]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2024-06-30 16:05:28,771 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:28,772 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-06-30 16:05:28,773 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ('spongebob', 'Spongebob Squarepants')
2024-06-30 16:05:28,774 INFO sqlalchemy.engine.Engine COMMIT


INSERT 语句不返回任何行，如果只插入一行，它通常会包含返回有关在插入该行期间生成的列级默认值的信息的能力，最常见的是整数主键值。在上面的情况下，SQLite 数据库中的第一行通常会为第一个整数主键值返回 1

In [26]:
result.inserted_primary_key

(1,)

In [27]:
print(insert(user_table)) # 自动生成values子句

INSERT INTO user_account (id, name, fullname) VALUES (:id, :name, :fullname)


In [28]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2024-06-30 16:05:31,240 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:05:31,242 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2024-06-30 16:05:31,243 INFO sqlalchemy.engine.Engine [generated in 0.00252s] [('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star')]
2024-06-30 16:05:31,244 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
#如果我们指示Insert.values()不带任何参数，则会生成一个真正的“空”INSERT，它只插入表的“默认值”，而不包含任何显式值
print(insert(user_table).values().compile(engine))

INSERT INTO user_account DEFAULT VALUES


In [30]:
insert_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [31]:
# 基于 INSERT…FROM SELECT 
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


### SELECT

In [35]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [36]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2024-06-30 16:06:21,846 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:06:21,847 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-06-30 16:06:21,848 INFO sqlalchemy.engine.Engine [generated in 0.00151s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2024-06-30 16:06:21,849 INFO sqlalchemy.engine.Engine ROLLBACK


在使用 ORM 时，特别是使用针对 ORM 实体构建的 select() 构造时，我们希望使用 Session.execute() 方法在 Session 上执行它；

使用这种方法，我们继续从结果中获取 Row 对象，但是这些行现在能够包含完整的实体，例如 User 类的实例，作为每行中的单个元素

In [37]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2024-06-30 16:06:23,289 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:06:23,291 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-06-30 16:06:23,292 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2024-06-30 16:06:23,294 INFO sqlalchemy.engine.Engine ROLLBACK


In [38]:
print(select(user_table.c.name, user_table.c.fullname))
print(select(user_table))
print(select(user_table.c["name", "fullname"]))
print(select(User))
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
SELECT user_account.name, user_account.fullname 
FROM user_account


In [39]:
row = session.execute(select(User)).first()
row

2024-06-30 16:06:24,995 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:06:24,996 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-06-30 16:06:24,997 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()


(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)

In [40]:
user = session.scalars(select(User)).first() #直接执行语句
user

2024-06-30 16:06:26,360 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-06-30 16:06:26,361 INFO sqlalchemy.engine.Engine [cached since 1.364s ago] ()


User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [41]:
row = session.execute(select(User.name, User.fullname)).first()
row

2024-06-30 16:06:26,882 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2024-06-30 16:06:26,884 INFO sqlalchemy.engine.Engine [generated in 0.00097s] ()


('spongebob', 'Spongebob Squarepants')

In [42]:
session.execute(
    select(User.name, Address).where(User.id == Address.user_id).order_by(Address.id)
).all()

2024-06-30 16:06:28,651 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2024-06-30 16:06:28,652 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ()


[]

In [43]:
from sqlalchemy import func, cast
stmt = select(
    ("Username: " + user_table.c.name).label("username"),
).order_by(user_table.c.name)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")

2024-06-30 16:06:31,927 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:06:31,928 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2024-06-30 16:06:31,929 INFO sqlalchemy.engine.Engine [generated in 0.00151s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2024-06-30 16:06:31,930 INFO sqlalchemy.engine.Engine ROLLBACK


In [44]:
from sqlalchemy import text
stmt = select(text("'some phrase'"), user_table.c.name).order_by(user_table.c.name)
with engine.connect() as conn:
    print(conn.execute(stmt).all())

2024-06-30 16:06:32,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:06:32,933 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2024-06-30 16:06:32,934 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ()
[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
2024-06-30 16:06:32,935 INFO sqlalchemy.engine.Engine ROLLBACK


In [45]:
from sqlalchemy import literal_column
stmt = select(literal_column("'some phrase'").label("p"), user_table.c.name).order_by(
    user_table.c.name
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")

2024-06-30 16:06:33,780 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:06:33,781 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2024-06-30 16:06:33,781 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ()
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2024-06-30 16:06:33,783 INFO sqlalchemy.engine.Engine ROLLBACK


In [46]:
print(user_table.c.name == "squidward")

print(address_table.c.user_id > 10)
print(select(user_table).where(user_table.c.name == "squidward"))
print(
    select(address_table.c.email_address)
    .where(user_table.c.name == "squidward")
    .where(address_table.c.user_id == user_table.c.id)
)
print(
    select(address_table.c.email_address).where(
        user_table.c.name == "squidward",
        address_table.c.user_id == user_table.c.id,
    )
)
from sqlalchemy import and_, or_
print(
    select(Address.email_address).where(
        and_(
            or_(User.name == "squidward", User.name == "sandy"),
            Address.user_id == User.id,
        )
    )
)
print(select(User).filter_by(name="spongebob", fullname="Spongebob Squarepants"))

user_account.name = :name_1
address.user_id > :user_id_1
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1
SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id
SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id
SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


In [47]:
print(select(user_table.c.name, address_table.c.email_address))
# ON 子句是推断出来的。
print(
    select(user_table.c.name, address_table.c.email_address).join_from(
        user_table, address_table
    )
)
print(select(user_table.c.name, address_table.c.email_address).join(address_table))
print(select(address_table.c.email_address).select_from(user_table).join(address_table))
from sqlalchemy import func
print(select(func.count("*")).select_from(user_table))
print(
    select(address_table.c.email_address)
    .select_from(user_table)
    .join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT user_account.name, address.email_address 
FROM user_account, address
SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
SELECT count(:count_2) AS count_1 
FROM user_account
SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [48]:
print(select(user_table).join(address_table, isouter=True)) # 外连接
print(select(user_table).join(address_table, full=True)) # 全连接

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


In [49]:
print(select(user_table).order_by(user_table.c.name))
print(select(User).order_by(User.fullname.desc()))

from sqlalchemy import func
count_fn = func.count(user_table.c.id)
print(count_fn)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.fullname DESC
count(user_account.id)


In [50]:
with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("count"))
        .join(Address)
        .group_by(User.name)
        .having(func.count(Address.id) > 1)
    )
    print(result.all())

2024-06-30 16:06:39,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-30 16:06:39,601 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > ?
2024-06-30 16:06:39,601 INFO sqlalchemy.engine.Engine [generated in 0.00164s] (1,)
[]
2024-06-30 16:06:39,603 INFO sqlalchemy.engine.Engine ROLLBACK


In [51]:
from sqlalchemy import func, desc
stmt = (
    select(Address.user_id, func.count(Address.id).label("num_addresses"))
    .group_by("user_id")
    .order_by("user_id", desc("num_addresses"))
)
print(stmt)

SELECT address.user_id, count(address.id) AS num_addresses 
FROM address GROUP BY address.user_id ORDER BY address.user_id, num_addresses DESC


In [52]:
user_alias_1 = user_table.alias()
user_alias_2 = user_table.alias()
print(
    select(user_alias_1.c.name, user_alias_2.c.name).join_from(
        user_alias_1, user_alias_2, user_alias_1.c.id > user_alias_2.c.id
    )
)

SELECT user_account_1.name, user_account_2.name AS name_1 
FROM user_account AS user_account_1 JOIN user_account AS user_account_2 ON user_account_1.id > user_account_2.id


In [53]:
# ORM 实体别名
from sqlalchemy.orm import aliased
address_alias_1 = aliased(Address)
address_alias_2 = aliased(Address)
print(
    select(User)
    .join_from(User, address_alias_1)
    .where(address_alias_1.email_address == "patrick@aol.com")
    .join_from(User, address_alias_2)
    .where(address_alias_2.email_address == "patrick@gmail.com")
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address = :email_address_1 AND address_2.email_address = :email_address_2


In [54]:
subq = (
    select(func.count(address_table.c.id).label("count"), address_table.c.user_id)
    .group_by(address_table.c.user_id)
    .subquery()
)
print(subq)

SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id
